In [2]:
import branca
import dask.distributed
import folium
import folium.plugins
import geopandas as gpd
import numpy as np
import rasterio as rio
import rasterio.features
import shapely.geometry
import stackstac
import xrspatial.multispectral as ms
from branca.element import Element, Figure
from odc.stac import configure_rio, stac_load
from pystac_client import Client

from utils import convert_bounds, image_on_map

In [ ]:
# start the dask cluster
client = dask.distributed.Client()
configure_rio(cloud_defaults=True, aws={"aws_unsigned": True}, client=client)
client

In [ ]:
# paste here the bbox json
aoi = {
    "type": "Polygon",
    "coordinates": [
        [
            [2.866745, 39.104489],
            [2.866745, 39.229594],
            [3.0336, 39.229594],
            [3.0336, 39.104489],
            [2.866745, 39.104489],
        ]
    ],
}
bbox = rasterio.features.bounds(aoi)

In [ ]:
catalog = Client.open("https://earth-search.aws.element84.com/v0")

query = catalog.search(
    collections=["sentinel-s2-l2a-cogs"],
    datetime="2022-4-1/2022-11-22",  # from / to
    bbox=bbox,
    query={"eo:cloud_cover": {"lt": 25}},
)

items = list(query.get_items())
print(f"Found: {len(items)} datasets")

# Convert STAC items into a GeoJSON FeatureCollection
stac_json = query.get_all_items_as_dict()

In [ ]:
data = (
    stackstac.stack(
        items,
        assets=["B04", "B03", "B02"],  # red, green, blue
        chunksize="auto",
        resolution=10,
        bounds_latlon=bbox,
        # epsg=3857
    )
    # .where(lambda x: x > 0, other=np.nan)  # sentinel-2 uses 0 as nodata
    .assign_coords(band=lambda x: x.common_name.rename("band"))  # use common names
)
data

In [ ]:
# use ms.true_color to convert RGB and improve the color
selection = data.isel(time=11)
image_on_map(ms.true_color(*selection).compute(), bbox)

In [ ]:
%%time
median = data.median(dim="time", keep_attrs=True).compute()

In [ ]:
image_on_map(ms.true_color(*median), bbox)

In [ ]:
median_color = ms.true_color(*median)

In [ ]:
median_color.transpose("band", "y", "x").rio.to_raster("images/cabrera_rgba.tif")